In [2]:
!python3 -m pip install beautifulsoup4 tqdm lxml pandas unidecode
# alternative to selenium
# !python3 -m pip install playwright


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [7]:
import os
import numpy as np
import pandas as pd
from datetime import date 
import time
import pickle
import json
import re
import requests
from urllib.error import HTTPError
import urllib.request
from bs4 import BeautifulSoup
import warnings
from pprint import pprint
warnings.filterwarnings('ignore')

In [12]:
import requests 
cookies = {
    '_ga': 'GA1.1.1636864037.1684958837',
    'trc_cookie_storage': 'taboola%2520global%253Auser-id%3Dd8ff65a6-5179-446b-acdf-7a5a5d36e76d-tuct965d9be',
    '_ga_0VVD9VP56V': 'GS1.1.1684958837.1.1.1684958896.1.0.0',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    # 'Cookie': '_ga=GA1.1.1636864037.1684958837; trc_cookie_storage=taboola%2520global%253Auser-id%3Dd8ff65a6-5179-446b-acdf-7a5a5d36e76d-tuct965d9be; _ga_0VVD9VP56V=GS1.1.1684958837.1.1.1684958896.1.0.0',
    'Referer': 'https://shoesoflasso.com/asic-mexico-66-kill-bill-of-maximilian-osinski-as-zava-in-ted-lasso-s03e03-4-5-1-2023/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="113", "Chromium";v="113", "Not-A.Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}
url = 'https://shoesoflasso.com/episode-list/'
r = requests.get(url, cookies=cookies, headers=headers) 
soup = BeautifulSoup(r.content) 
# print(soup.prettify()) 
web_links = soup.select('a') 
actual_web_links = [web_link['href'] for web_link in web_links] 
actual_web_links

['https://shoesoflasso.com/',
 'https://shoesoflasso.com/',
 'https://shoesoflasso.com/episode-list/',
 'https://shoesoflasso.com/season-1-shoes-in-ted-lasso/',
 'https://shoesoflasso.com/pilot-season-1-episode-1/',
 'https://shoesoflasso.com/biscuits-season-1-episode-2/',
 'https://shoesoflasso.com/trent-crimm-the-independent-season-1-episode-3/',
 'https://shoesoflasso.com/tan-lines-new-underwear-season-1-episode-5/',
 'https://shoesoflasso.com/two-aces-season-1-episode-6/',
 'https://shoesoflasso.com/make-rebecca-great-again-season-1-episode-7/',
 'https://shoesoflasso.com/the-diamond-dogs-season-1-episode-8/',
 'https://shoesoflasso.com/all-apologies-season-1-episode-9/',
 'https://shoesoflasso.com/the-hope-that-kills-you-season-1-episode-10/',
 'https://shoesoflasso.com/season-2-shoes-in-ted-lasso/',
 'https://shoesoflasso.com/ted-lasso-goodbye-earl-season-2-episode-1/',
 'https://shoesoflasso.com/lavender-season-2-episode-2/',
 'https://shoesoflasso.com/ted-lasso-do-the-right-est

In [15]:
import re

def filter_urls(urls):
  """
  Filters a list of URL strings using regex such that each link must end with
  "season" followed by "-" followed by a number followed by "-" followed by
  "episode" followed by "-" followed by a number followed by "/".

  Args:
    urls: A list of URL strings.

  Returns:
    A list of filtered URL strings.
  """

  # Create a regular expression to match the desired pattern.
  pattern = re.compile(r'.*season-\d+-ep-\d+\/')

  # Filter the list of URL strings.
  filtered_urls = [url for url in urls if re.match(pattern, url)]

  return filtered_urls

episodes_w_sneaks = filter_urls(actual_web_links)
print(f"Found {len(episodes_w_sneaks)} Ted Lasso episodes with sneakers!")
print(episodes_w_sneaks)

Found 30 Ted Lasso episodes with sneakers!
['https://shoesoflasso.com/no-weddings-and-a-funeral-season-2-ep-10/', 'https://shoesoflasso.com/midnight-train-to-royston-season-2-ep-11/', 'https://shoesoflasso.com/inverting-the-pyramid-of-success-season-2-ep-12/', 'https://shoesoflasso.com/smells-like-mean-spirit-season-3-ep-1/', 'https://shoesoflasso.com/i-dont-want-to-go-to-chelsea-season-3-ep-2/', 'https://shoesoflasso.com/4-5-1-season-3-ep-3/', 'https://shoesoflasso.com/big-week-season-3-ep-4/', 'https://shoesoflasso.com/signs-season-3-ep-5/', 'https://shoesoflasso.com/sunflowers-season-3-ep-6/', 'https://shoesoflasso.com/the-strings-that-bind-us-season-3-ep-7/', 'https://shoesoflasso.com/well-never-have-paris-season-3-ep-8/', 'https://shoesoflasso.com/no-weddings-and-a-funeral-season-2-ep-10/', 'https://shoesoflasso.com/midnight-train-to-royston-season-2-ep-11/', 'https://shoesoflasso.com/inverting-the-pyramid-of-success-season-2-ep-12/', 'https://shoesoflasso.com/smells-like-mean-spi

In [20]:
def scrape_sneaks(url):
    global cookies, headers
    r = requests.get(url, cookies=cookies, headers=headers) 
    soup = BeautifulSoup(r.content)
    web_links = soup.findAll('a', attrs={'class': 'eael-grid-post-link'}) 
    print(web_links)

for url in episodes_w_sneaks:
    scrape_sneaks(url)
    break

[<a class="eael-grid-post-link" href="https://shoesoflasso.com/new-balance-shoes-worn-by-brendan-hunt-as-coach-beard-in-ted-lasso-s02e10-no-weddings-and-a-funeral-2021/" title="New Balance Shoes Worn by Brendan Hunt as Coach Beard in Ted Lasso S02E10 “No Weddings and a Funeral” (2021)">New Balance Shoes Worn by Brendan Hunt as Coach Beard in Ted Lasso S02E10 “No Weddings and a Funeral” (2021)</a>]
